In [2]:
from typing import List
from IPython.display import display, Math, Latex

def bb(n, bits=7):
    mask = (1 << bits) - 1
    if n < 0:
        n = ((abs(n) ^ mask) + 1)
    return ('{0:0' + str(bits) + 'b}').format(n & mask)
  
def bn(n, bits=7, underline=None, sep=None):
    mask = (1 << bits) - 1
    if n < 0:
        n = ((abs(n) ^ mask) + 1)
    arr = list(('{0:0' + str(bits) + 'b}').format(n & mask))
    
    arr.reverse()
    if underline is not None: arr[underline] = r'\underline{' + arr[underline] + '}'
    if sep is not None: arr.insert(sep, '|')
    arr.reverse()
    return '\ '.join(arr)
 

def text(s: str) -> str:
  s = s.replace(' ', '\ ')
  s = r'\textup{' + s + '}'
  return s

def let(a: str, sign = 0, sub = '') -> str:
  r = r' \left[' + ('-' if sign else '') + a + r'\right]_{\textup{' + sub + r'}}'
  return r

def arr(l: List[str], t='c') -> str:
  r = r'\begin{array}{' + t + r'}'
  r += r' \\ '.join(l)
  r += r'\end{array}'
  return r

def fix(val, digits=16):
  # return int(bb(val, digits), 2)
  val &= ((1<<digits)-1)
  if val & (1<<(digits - 1)): val -= (1<<digits)
  return val

def sign(n: int):
  if n < 0: return 1
  return 0

class Mul2():
  table: List[List[str]]
  
  def __init__(self, a: int, b: int) -> None:
    self.a = a
    self.b = b
    self.table = [[
        arr([
          text('Операнды'),
        ]),
        arr([
          text('СЧП (старшие разряды)'),
        ]),
        arr([
          text(' В/СЧП (младшие разряды)'),
        ]),
        arr([
          text('Признак коррекции'),
        ])
    ]]
    self.top = ''
    self.bot = ''
    self.process(a, b)
  
  def __str__(self) -> str:
    return self.print()
    
  def print(self):
    la = r'[A]_{пр.}' if self.a > 0 else  r'[-A]_{доп.}'
    lb = r'[B]_{пр.}' if self.b > 0 else  r'[-B]_{доп.}'
    la += '=' + bb(self.a)
    lb += '=' + bb(self.b)
    self.top = (
      '(A' + ('>' if self.a > 0 else '<') + '0,\ B' + ('>' if self.b > 0 else '<') + '0)')
    self.top += r' \\ ' + '\n'
    self.top += '\left.' + la + ';\ ' + lb + '\\right.'
    
    res = r'\begin{array}{c}'
    # res += self.top
    # res += r' \\ '  + '\n' + r' \\ '
    res += (
      r'\begin{array}{|c|c|c|c|} \hline ' +
      (r' \\ \hline ' + '\n').join(' & '.join(row) for row in self.table) +
      r' \\ \hline ' + '\n' + r' \end{array} \\' + '\n'
      )
    res += r' \\ ' + '\n' + r' \\ '
    res += self.bot
    res += r'\end{array}'
    
    return f'$${res}$$'
    # display(Math(res))
    
  def process(self, a: int, b: int):
    s = b
    self.table.append([
      text('СЧП'),
      bn(s >> 12, 15),
      bn(s, 12),
      str(0)
    ])
    cor = 0
    for i in range(6):
      t = []
      r = bb(s, 12)[-2:]
      ss = 0
      l = ''
      print(r)
      if cor == 0:
        if r == '00':
          ss, l, cor = 0, '0', 0
        elif r == '01':
          ss, l, cor = a, 'M_A', 0
        elif r == '10':
          ss, l, cor = a + a, '2M_A', 0
        else:
          ss, l, cor = - a, '-M_A', 1
      else:
        if r == '00':
          ss, l, cor = + a, 'M_A', 0
        elif r == '01':
          ss, l, cor = a + a, '2M_A', 0
        elif r == '10':
          ss, l, cor = - a, '-M_A', 1
        else:
          ss, l, cor = s, '0', 1
      self.table.append([
        arr([
          let(l),
            text('СЧП'),
              text('СЧП \\rightarrow 2'),
        ]),
        arr([
          bn((ss    ), 15),
          bn(((ss << 12) + s) >> 12, 15),
          bn(((ss << 12) + s) >> 14, 15)
        ]),
        arr([
          '',
          bn(((ss << 12) + s), 12),
          bn(((ss << 12) + s) >> 2, 12)
        ]),
        str(cor)
      ])
      s = s + (ss << 12)
      s = s >> 2
    
    print(hex(int(bb(s >> 12, 12), 2)))
    # self.bot = r' \left[C\right]_{\textup{пр.}}' if a > 0 else  r' \left[C\right]_{\textup{доп}.}'
    # self.bot += '='
    # self.bot += bb( a * b, aw * 2) + '_2 = '
    # self.bot += str(a * b) + '_{10}'
    
open('res2.tex', 'w+', encoding='utf-8').write(f'''
  {Mul2(0b100011100110, (0b000100100011))}
''')
# open('res.tex', 'w+', encoding='utf-8').write(f'''
#   {Mul2(0b011101001101, 0b100011001101)}
# ''')
0b010000000001


11
00
10
00
01
00
0xa1


1025